In [1]:
import numpy as np
import plotly.graph_objects as go
from sklearn.cluster import DBSCAN
from sklearn.preprocessing import StandardScaler
from scipy.optimize import least_squares
from scipy.optimize import brentq
import matplotlib.pyplot as plt

import pandas as pd
pd.set_option('display.max_rows', 100)      # Max 100 Zeilen anzeigen
pd.set_option('display.max_columns', None)  # Alle Spalten anzeigen
pd.set_option('display.width', 1000)        # Zeilenbreite erhöhen, damit kein Umbruch erfolgt
pd.set_option('display.float_format', '{:.4f}'.format) # 4 Nachkommastellen für die Optik

import sys
from pathlib import Path
PROJECT_ROOT = Path().resolve().parents[0]  # notebooks/ → cone-operator-lab/
sys.path.insert(0, str(PROJECT_ROOT))

from modules.hearing_box import compute_A2_hat

In [2]:
path = r"C:/Users/sulta/git/cone-operator-lab/data/meshes/stability_surface_box_abcxyz.csv"
df = pd.read_csv(path)

In [3]:
# Transformation berechnen
df['u'] = (np.log(df['b']/df['a']) + np.log(df['c']/df['a'])) / np.sqrt(2)
df['v'] = (np.log(df['b']/df['a']) - np.log(df['c']/df['a'])) / np.sqrt(2)

# 2. Isolieren des Plateaus (z ~ 69)
plateau = df[df['z'].between(68.5, 69.5)].copy()

In [4]:
# Skalierung für DBSCAN (u und v haben oft unterschiedliche Varianzen)
scaler = StandardScaler()
uv_scaled = scaler.fit_transform(plateau[['u', 'v']])

# eps muss im u/v Raum evtl. angepasst werden (0.2 - 0.4 ist oft gut)
db = DBSCAN(eps=0.25, min_samples=5).fit(uv_scaled)
plateau['cluster'] = db.labels_

# Wir filtern Noise (-1) aus, um nur die sauberen Cluster zu zeigen
clusters_only = plateau[plateau['cluster'] != -1]
plateau69 = plateau[plateau['cluster'] == 0].copy()


In [5]:
def egg_S_uv(u, v, u0, a_ei, b_ei, kappa):
    # denom_u must stay positive -> use a_ei>0, and constrain (1 + kappa*(u-u0)) > 0
    denom_u = (a_ei * (1.0 + kappa * (u - u0)))**2
    denom_v = b_ei**2
    return ((u - u0)**2 / denom_u) + (v**2 / denom_v)

def R_of_theta(theta, u0, a_ei, b_ei, kappa, Rmax=10.0):
    c = np.cos(theta)
    s = np.sin(theta)

    def F(R):
        guard = 1.0 + kappa*(R*c)
        if guard <= 0:
            return 1e6
        S = (R**2 * c**2) / (a_ei**2 * guard**2) + (R**2 * s**2) / (b_ei**2)
        return S - 1.0

    # bracket root
    lo, hi = 0.0, 1e-6
    while F(hi) < 0 and hi < Rmax:
        hi *= 2
    if hi >= Rmax:
        return np.nan
    return brentq(F, 0.0, hi)


In [6]:
U = plateau69["u"].to_numpy()
V = plateau69["v"].to_numpy()

# Center for initial guess
u0_init, v0_init = U.mean(), V.mean()

du, dv = U - u0_init, V - v0_init
theta = np.arctan2(dv, du)
r = np.sqrt(du**2 + dv**2)

# boundary by angle bin (max r per bin)
n_bins = 180
bins = np.linspace(-np.pi, np.pi, n_bins + 1)
bin_id = np.digitize(theta, bins) - 1

theta_b, r_b = [], []
for k in range(n_bins):
    m = bin_id == k
    if m.sum() < 2:
        continue
    theta_b.append(0.5*(bins[k]+bins[k+1]))
    r_b.append(r[m].max())

theta_b = np.array(theta_b)
r_b = np.array(r_b)

u_b = u0_init + r_b*np.cos(theta_b)
v_b = v0_init + r_b*np.sin(theta_b)

# Initial guesses for ellipse-ish parameters
a_init = (u_b.max() - u_b.min()) / 2
b_init = (np.abs(v_b).max())
kappa_init = 0.0

p0 = np.array([u0_init, a_init, b_init, kappa_init], dtype=float)

def residuals(p):
    u0, a_ei, b_ei, kappa = p
    # Penalty if denom_u can flip sign (keep 1 + kappa*(u-u0) positive)
    guard = 1.0 + kappa*(u_b - u0)
    if np.any(guard <= 0):
        return 1e3 * np.ones_like(u_b)
    S = egg_S_uv(u_b, v_b, u0, a_ei, b_ei, kappa)
    return S - 1.0

# Bounds: a_ei>0, b_ei>0; kappa typically small
lb = np.array([-np.inf, 1e-6, 1e-6, -2.0])
ub = np.array([ np.inf,  np.inf,  np.inf,  2.0])

res = least_squares(residuals, p0, bounds=(lb, ub))
u0_fit, a_fit, b_fit, kappa_fit = res.x

u0_fit, a_fit, b_fit, kappa_fit, res.cost

theta_s = np.linspace(-np.pi, np.pi, 600)
R_s = np.array([R_of_theta(t, u0_fit, a_fit, b_fit, kappa_fit) for t in theta_s])

u_egg = u0_fit + R_s*np.cos(theta_s)
v_egg = 0.0 + R_s*np.sin(theta_s)

In [7]:
print("\n=== Egg model (S = 1) ===\n")
print(f"u0     = {u0_fit:.8f}")
print(f"a_ei   = {a_fit:.8f}")
print(f"b_ei   = {b_fit:.8f}")
print(f"kappa  = {kappa_fit:.8f}")

print("\nImplicit curve equation:\n")
print(
    f"((u - {u0_fit:.6f})^2) / "
    f"({a_fit:.6f}^2 * (1 + {kappa_fit:.6f}*(u - {u0_fit:.6f}))^2) "
    f"+ (v^2 / {b_fit:.6f}^2) = 1"
)

latex_formula = (
    r"\frac{(u - %.6f)^2}{(%.6f\,(1 + %.6f\,(u - %.6f)))^2}"
    r"+ \frac{v^2}{%.6f^2} = 1"
    % (u0_fit, a_fit, kappa_fit, u0_fit, b_fit)
)

print(latex_formula)


=== Egg model (S = 1) ===

u0     = 0.23927385
a_ei   = 0.92724206
b_ei   = 0.60125043
kappa  = -0.33959903

Implicit curve equation:

((u - 0.239274)^2) / (0.927242^2 * (1 + -0.339599*(u - 0.239274))^2) + (v^2 / 0.601250^2) = 1
\frac{(u - 0.239274)^2}{(0.927242\,(1 + -0.339599\,(u - 0.239274)))^2}+ \frac{v^2}{0.601250^2} = 1


In [19]:
fig = go.Figure()

# Background: full f*=69 plateau (visible but unobtrusive)
fig.add_trace(go.Scatter(
    x=plateau['u'],
    y=plateau['v'],
    mode='markers',
    marker=dict(
        color='rgba(120,120,120,0.20)',
        size=8,
        line=dict(width=0)
    ),
    name='Plateau points (f* = 69)',
    hoverinfo='skip'
))

mask = np.isfinite(u_egg) & np.isfinite(v_egg)

fig.add_trace(go.Scatter(
    x=u_egg[mask],
    y=v_egg[mask],
    mode="lines",
    line=dict(
        width=1.5,
        color="#ff0000"
    ),
    name="Egg fit of the f* = 69 contour",
    hoverinfo="skip"
))

fig.update_layout(
    template="plotly_white",
    width=800,
    height=700,
    showlegend=False,
    legend=dict(
        title="",
        x=0.02, y=0.98,
        xanchor="left", yanchor="top",
        bgcolor="rgba(255,255,255,0.8)",
        bordercolor="rgba(0,0,0,0.15)",
        borderwidth=1
    ),
    margin=dict(l=90, r=25, t=20, b=80),
    xaxis=dict(
        title="u = (log x + log y) / √2",
        scaleanchor="y",
        scaleratio=1,
        gridcolor='whitesmoke',
        zeroline=False,
        ticks="outside",
        ticklen=6,
        tickwidth=1
    ),
    yaxis=dict(
        title="v = (log x − log y) / √2",
        gridcolor='whitesmoke',
        zeroline=False,
        ticks="outside",
        ticklen=6,
        tickwidth=1
    )
)

fig.show()

In [12]:
# --- 2) Compute S and verification error ---
plateau69["S_model"] = egg_S_uv(
    plateau69["u"].to_numpy(),
    plateau69["v"].to_numpy(),
    u0_fit, a_fit, b_fit, kappa_fit
)

plateau69["S_err"] = plateau69["S_model"] - 1.0
plateau69["abs_S_err"] = plateau69["S_err"].abs()

# --- 3) Summary stats (quality report) ---
print("=== Verification on z≈69 plateau only ===")
print("n points:", len(plateau69))
print("mean(S-1):", plateau69["S_err"].mean())
print("std(S-1): ", plateau69["S_err"].std())
print("median |S-1|:", plateau69["abs_S_err"].median())
print("90% |S-1|:   ", plateau69["abs_S_err"].quantile(0.90))
print("95% |S-1|:   ", plateau69["abs_S_err"].quantile(0.95))
print("max  |S-1|:  ", plateau69["abs_S_err"].max())

rmse = np.sqrt(np.mean((plateau69["S_model"] - 1.0)**2))
print("RMSE on plateau:", rmse)

# Auswahl von 100 Zufallsstichproben
n_samples = min(100, len(plateau69))
sample_df = plateau69[['a', 'b', 'c', 'z', 'S_model']].sample(n_samples).copy()

# 3. Zielwert und Abweichung berechnen
sample_df['z_target'] = 69.0
sample_df['Abw_z'] = sample_df['z'] - sample_df['z_target']

# 4. Ausgabe
print(f"\n--- VERIFIKATIONSTABELLE: {n_samples} STICHPROBEN AUS CLUSTER 0 ---")
print(sample_df[['a', 'b', 'c', 'z', 'z_target', 'Abw_z', 'S_model']].to_string(index=False))

=== Verification on z≈69 plateau only ===
n points: 317
mean(S-1): -0.009410256247255255
std(S-1):  0.009540876213207048
median |S-1|: 0.009992913846936125
90% |S-1|:    0.020246942322401942
95% |S-1|:    0.025944963562949282
max  |S-1|:   0.03433605672940221
RMSE on plateau: 0.013390074158665843

--- VERIFIKATIONSTABELLE: 100 STICHPROBEN AUS CLUSTER 0 ---
     a      b      c  z  z_target  Abw_z  S_model
1.0000 2.0717 1.8241 69   69.0000 0.0000   1.0060
1.0000 1.1091 2.2726 69   69.0000 0.0000   0.9823
1.0000 1.3659 2.3258 69   69.0000 0.0000   0.9935
1.0000 2.2726 1.0963 69   69.0000 0.0000   0.9932
1.0000 0.4344 0.6513 69   69.0000 0.0000   1.0044
1.0000 1.7618 2.1202 69   69.0000 0.0000   1.0012
1.0000 2.3258 1.3346 69   69.0000 0.0000   0.9869
1.0000 1.8241 0.7838 69   69.0000 0.0000   0.9870
1.0000 0.4766 0.8499 69   69.0000 0.0000   0.9953
1.0000 0.4934 0.9110 69   69.0000 0.0000   0.9847
1.0000 1.4306 2.3258 69   69.0000 0.0000   1.0173
1.0000 0.5669 1.1888 69   69.0000 0.0000 